In [98]:
import pandas as pd
import numpy as np
import psycopg2
from sqlalchemy.engine.create import create_engine
import os

In [99]:
# This notebook will be mainly used for cleaning and performing high level analysis of school testing data for Tennessee schools for the 2021-2022, 2020-2021, 2017-2018, and 2012-2013 schoo years.


In [100]:
# Import the xslx/csv for each school year and convert to a dataframe
read_sc22_df = pd.read_excel('Data/2022/school_assessment_file_suppressed_upd32323 (1).xlsx')
read_sc21_df = pd.read_csv('Data/2021/school_assessment_file_suppressed_upd422.csv')
read_sc18_df = pd.read_csv('Data/2018/data_2018_school_base.csv')
read_sc13_df = pd.read_excel('Data/2013/data_2013_school_base.xlsx')

sc13_df = read_sc13_df
sc18_df = read_sc18_df
sc21_df = read_sc21_df
sc22_df = read_sc22_df

In [101]:
# Check data types
sc22_df.info()
sc21_df.info()
sc18_df.info()
sc13_df.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 675534 entries, 0 to 675533
Data columns (total 22 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   year                       675534 non-null  int64 
 1   system                     675534 non-null  int64 
 2   system_name                675534 non-null  object
 3   school                     675534 non-null  int64 
 4   school_name                675534 non-null  object
 5   test                       675534 non-null  object
 6   subject                    675534 non-null  object
 7   grade                      675534 non-null  object
 8   student_group              675534 non-null  object
 9   participation_rate         675534 non-null  object
 10  enrolled                   675534 non-null  int64 
 11  tested                     675534 non-null  int64 
 12  valid_tests                675534 non-null  int64 
 13  n_below                    675534 non-null  

In [102]:
# Asterisks represent negligible values in the scope of the data. Replace '*' and '**' with 0 for all columns
# sc22_df.replace(['*', '**'], 0, inplace=True)
# sc21_df.replace(['*', '**'], 0, inplace=True)


In [103]:
# Check if the number of school systems changed across the years
sc22_df['system_name'].nunique()

147

In [104]:
sc21_df['system_name'].nunique()

147

In [105]:
# Check if the number of schools changed across the years
sc22_df['school_name'].nunique()

1716

In [106]:
# Check if the number of schools changed across the years
sc21_df['school_name'].nunique()

1671

In [107]:
# Check for null values
sc22_df.isnull().sum()

year                         0
system                       0
system_name                  0
school                       0
school_name                  0
test                         0
subject                      0
grade                        0
student_group                0
participation_rate           0
enrolled                     0
tested                       0
valid_tests                  0
n_below                      0
n_approaching                0
n_met_expectations           0
n_exceeded_expectations      0
pct_below                    0
pct_approaching              0
pct_met_expectations         0
pct_exceeded_expectations    0
pct_met_exceeded             0
dtype: int64

In [108]:
# Check for null values in 2021
sc21_df.isnull().sum()

year                  0
system                0
system_name           0
school                0
school_name           0
test                  0
subject               0
grade                 0
subgroup              0
participation_rate    0
enrolled              0
tested                0
valid_tests           0
n_below               0
n_approaching         0
n_on_track            0
n_mastered            0
pct_below             0
pct_approaching       0
pct_on_track          0
pct_mastered          0
pct_on_mastered       0
dtype: int64

In [109]:
# List unique values in student_group
sc22_df['student_group'].nunique()

22

In [110]:
sc21_df['subgroup'].nunique()

21

In [111]:
# Create a new DataFrame with column headers from sc21_df and sc22_df
column_comparison_df = pd.DataFrame({'sc21_Columns': sc21_df.columns, 'sc22_Columns': sc22_df.columns})

print(column_comparison_df)


          sc21_Columns               sc22_Columns
0                 year                       year
1               system                     system
2          system_name                system_name
3               school                     school
4          school_name                school_name
5                 test                       test
6              subject                    subject
7                grade                      grade
8             subgroup              student_group
9   participation_rate         participation_rate
10            enrolled                   enrolled
11              tested                     tested
12         valid_tests                valid_tests
13             n_below                    n_below
14       n_approaching              n_approaching
15          n_on_track         n_met_expectations
16          n_mastered    n_exceeded_expectations
17           pct_below                  pct_below
18     pct_approaching            pct_approaching


In [112]:
# Rename columns in sc21 to match sc22
sc21_df = sc21_df.rename(columns={'subgroup': 'student_group', 'n_on_track': 'n_met_expectations','n_mastered': 'n_exceeded_expectations', 'pct_on_track': 'pct_met_expectations','pct_mastered': 'pct_exceeded_expectations', 'pct_on_mastered': 'pct_met_exceeded'})
column_comparison_after_df = pd.DataFrame({'sc21_col': sc21_df.columns, 'sc22_col': sc22_df.columns})
print(column_comparison_after_df)


                     sc21_col                   sc22_col
0                        year                       year
1                      system                     system
2                 system_name                system_name
3                      school                     school
4                 school_name                school_name
5                        test                       test
6                     subject                    subject
7                       grade                      grade
8               student_group              student_group
9          participation_rate         participation_rate
10                   enrolled                   enrolled
11                     tested                     tested
12                valid_tests                valid_tests
13                    n_below                    n_below
14              n_approaching              n_approaching
15         n_met_expectations         n_met_expectations
16    n_exceeded_expectations  

In [113]:
# List the tests taken in 2022
sc22_df['subject'].unique()

array(['Algebra I', 'Algebra II', 'Biology I', 'English I', 'English II',
       'Geometry', 'US History', 'ELA', 'Math', 'Science',
       'Social Studies', 'Integrated Math I', 'Integrated Math II',
       'Integrated Math III'], dtype=object)

In [114]:
sc21_df['subject'].unique()

array(['Algebra I', 'Algebra II', 'Biology I', 'English I', 'English II',
       'Geometry', 'US History', 'ELA', 'Math', 'Science',
       'Social Studies', 'Integrated Math I', 'Integrated Math II',
       'Integrated Math III'], dtype=object)

In [115]:
# List the unique names of the grades in the df
sc22_df['grade'].unique()

array(['10', '11', '9', 'All Grades', 'Missing Grade', '12', '8', '4',
       '7', '3', '5', '6'], dtype=object)

In [116]:
sc21_df['grade'].unique()

array(['10', '11', '12', '9', 'All Grades', '5', '3', '4', '8', '6', '7',
       'Missing Grade'], dtype=object)

In [117]:
# Filter both df's to include only 9-12th graders
# Define the list of grades to keep
grades_to_keep = ['9', '10', '11', '12']

# Filter sc21_df and sc22_df to keep only rows with the specified grades
sc21_df = sc21_df[sc21_df['grade'].isin(grades_to_keep)]
sc22_df = sc22_df[sc22_df['grade'].isin(grades_to_keep)]

In [125]:
# Count the number of each test submitted in both the dataframes.
# 2021
alg1_total_tested_sc21 = sc21_df.loc[(sc21_df['student_group'] == 'All Students') & (sc21_df['subject'] == 'Algebra I'), 'tested'].sum()
alg2_total_tested_sc21 = sc21_df.loc[(sc21_df['student_group'] == 'All Students') & (sc21_df['subject'] == 'Algebra II'), 'tested'].sum()
bio1_total_tested_sc21 = sc21_df.loc[(sc21_df['student_group'] == 'All Students') & (sc21_df['subject'] == 'Biology I'), 'tested'].sum()
eng1_total_tested_sc21 = sc21_df.loc[(sc21_df['student_group'] == 'All Students') & (sc21_df['subject'] == 'English I'), 'tested'].sum()
eng2_total_tested_sc21 = sc21_df.loc[(sc21_df['student_group'] == 'All Students') & (sc21_df['subject'] == 'English II'), 'tested'].sum()
geom_total_tested_sc21 = sc21_df.loc[(sc21_df['student_group'] == 'All Students') & (sc21_df['subject'] == 'Geometry'), 'tested'].sum()
ush_total_tested_sc21 = sc21_df.loc[(sc21_df['student_group'] == 'All Students') & (sc21_df['subject'] == 'US History'), 'tested'].sum()
total_tests_sc21 = (
    alg1_total_tested_sc21
    + alg2_total_tested_sc21
    + bio1_total_tested_sc21
    + eng1_total_tested_sc21
    + eng2_total_tested_sc21
    + geom_total_tested_sc21
    + ush_total_tested_sc21
)


# 2022
alg1_total_tested_sc22 = sc22_df.loc[(sc22_df['student_group'] == 'All Students') & (sc22_df['subject'] == 'Algebra I'), 'tested'].sum()
alg2_total_tested_sc22 = sc22_df.loc[(sc22_df['student_group'] == 'All Students') & (sc22_df['subject'] == 'Algebra II'), 'tested'].sum()
bio1_total_tested_sc22 = sc22_df.loc[(sc22_df['student_group'] == 'All Students') & (sc22_df['subject'] == 'Biology I'), 'tested'].sum()
eng1_total_tested_sc22 = sc22_df.loc[(sc22_df['student_group'] == 'All Students') & (sc22_df['subject'] == 'English I'), 'tested'].sum()
eng2_total_tested_sc22 = sc22_df.loc[(sc22_df['student_group'] == 'All Students') & (sc22_df['subject'] == 'English II'), 'tested'].sum()
geom_total_tested_sc22 = sc22_df.loc[(sc22_df['student_group'] == 'All Students') & (sc22_df['subject'] == 'Geometry'), 'tested'].sum()
ush_total_tested_sc22 = sc22_df.loc[(sc22_df['student_group'] == 'All Students') & (sc22_df['subject'] == 'US History'), 'tested'].sum()
total_tests_sc22 = (
    alg1_total_tested_sc22
    + alg2_total_tested_sc22
    + bio1_total_tested_sc22
    + eng1_total_tested_sc22
    + eng2_total_tested_sc22
    + geom_total_tested_sc22
    + ush_total_tested_sc22
)

# Create a dictionary with the data
data = {
    'Subject': ['Algebra I', 'Algebra II', 'Biology I', 'English I', 'English II', 'Geometry', 'US History', 'Total'],
    'Total Tested 2021': [
        alg1_total_tested_sc21,
        alg2_total_tested_sc21,
        bio1_total_tested_sc21,
        eng1_total_tested_sc21,
        eng2_total_tested_sc21,
        geom_total_tested_sc21,
        ush_total_tested_sc21,
        total_tests_sc21
    ],
    'Total Tested 2022': [
        alg1_total_tested_sc22,
        alg2_total_tested_sc22,
        bio1_total_tested_sc22,  
        eng1_total_tested_sc22,  
        eng2_total_tested_sc22,  
        geom_total_tested_sc22,  
        ush_total_tested_sc22,
        total_tests_sc22  
    ],
}

# Create the DataFrame using the dictionary
tot_comparison_df = pd.DataFrame(data)

# Calculate the percent change
tot_comparison_df['Percent Change'] = (
    (tot_comparison_df['Total Tested 2022'] - tot_comparison_df['Total Tested 2021'])
    / tot_comparison_df['Total Tested 2021']
) * 100

# Format the percent change column with the desired number of significant figures and a '%' symbol
tot_comparison_df['Percent Change'] = tot_comparison_df['Percent Change'].apply(lambda x: f'{x:.1f}%')

# Display the updated DataFrame
tot_comparison_df



,Subject,Total Tested 2021,Total Tested 2022,Percent Change
0,Algebra I,51526,58434,13.4%
1,Algebra II,48716,53409,9.6%
2,Biology I,62925,73175,16.3%
3,English I,68868,75458,9.6%
4,English II,65391,71439,9.2%
5,Geometry,53279,57570,8.1%
6,US History,37652,39147,4.0%
7,Total,388357,428632,10.4%


In [133]:
# There's a large jump in test takers for Algebra I and Biology I. Let's dive deeper:
alg1_9_tested_sc21 = sc21_df.loc[(sc21_df['student_group'] == 'All Students') & (sc21_df['grade'] == '9') & (sc21_df['subject'] == 'Algebra I'), 'tested'].sum()
alg1_10_tested_sc21 = sc21_df.loc[(sc21_df['student_group'] == 'All Students') & (sc21_df['grade'] == '10') & (sc21_df['subject'] == 'Algebra I'), 'tested'].sum()
alg1_11_tested_sc21 = sc21_df.loc[(sc21_df['student_group'] == 'All Students') & (sc21_df['grade'] == '11') & (sc21_df['subject'] == 'Algebra I'), 'tested'].sum()
alg1_12_tested_sc21 = sc21_df.loc[(sc21_df['student_group'] == 'All Students') & (sc21_df['grade'] == '12') & (sc21_df['subject'] == 'Algebra I'), 'tested'].sum()

alg1_9_tested_sc22 = sc22_df.loc[(sc22_df['student_group'] == 'All Students') & (sc22_df['grade'] == '9') & (sc22_df['subject'] == 'Algebra I'), 'tested'].sum()
alg1_10_tested_sc22 = sc22_df.loc[(sc22_df['student_group'] == 'All Students') & (sc22_df['grade'] == '10') & (sc22_df['subject'] == 'Algebra I'), 'tested'].sum()
alg1_11_tested_sc22 = sc22_df.loc[(sc22_df['student_group'] == 'All Students') & (sc22_df['grade'] == '11') & (sc22_df['subject'] == 'Algebra I'), 'tested'].sum()
alg1_12_tested_sc22 = sc22_df.loc[(sc22_df['student_group'] == 'All Students') & (sc22_df['grade'] == '12') & (sc22_df['subject'] == 'Algebra I'), 'tested'].sum()


alg1_data = {
    'Algebra I': ['9th Grade', '10th Grade', '11th Grade', '12th Grade'], '2021': [alg1_9_tested_sc21, alg1_10_tested_sc21, alg1_11_tested_sc21, alg1_12_tested_sc21], 'Algebra I': ['9th Grade', '10th Grade', '11th Grade', '12th Grade'], '2022': [alg1_9_tested_sc22, alg1_10_tested_sc22, alg1_11_tested_sc22, alg1_12_tested_sc22]
}

alg1_testers_df = pd.DataFrame(alg1_data)

# Calculate the percentage change in test takers for Algebra I
alg1_testers_df['Percent Change'] = (alg1_testers_df['2022'] - alg1_testers_df['2021']) / alg1_testers_df['2021'] * 100
alg1_testers_df['Percent Change'] = alg1_testers_df['Percent Change'].apply(lambda x: f'{x:.1f}%')


alg1_testers_df


,Algebra I,2021,2022,Percent Change
0,9th Grade,47384,53262,12.4%
1,10th Grade,3699,4659,26.0%
2,11th Grade,320,396,23.8%
3,12th Grade,123,117,-4.9%


In [134]:
bio1_9_tested_sc21 = sc21_df.loc[(sc21_df['student_group'] == 'All Students') & (sc21_df['grade'] == '9') & (sc21_df['subject'] == 'Biology I'), 'tested'].sum()
bio1_10_tested_sc21 = sc21_df.loc[(sc21_df['student_group'] == 'All Students') & (sc21_df['grade'] == '10') & (sc21_df['subject'] == 'Biology I'), 'tested'].sum()
bio1_11_tested_sc21 = sc21_df.loc[(sc21_df['student_group'] == 'All Students') & (sc21_df['grade'] == '11') & (sc21_df['subject'] == 'Biology I'), 'tested'].sum()
bio1_12_tested_sc21 = sc21_df.loc[(sc21_df['student_group'] == 'All Students') & (sc21_df['grade'] == '12') & (sc21_df['subject'] == 'Biology I'), 'tested'].sum()

bio1_9_tested_sc22 = sc22_df.loc[(sc22_df['student_group'] == 'All Students') & (sc22_df['grade'] == '9') & (sc22_df['subject'] == 'Biology I'), 'tested'].sum()
bio1_10_tested_sc22 = sc22_df.loc[(sc22_df['student_group'] == 'All Students') & (sc22_df['grade'] == '10') & (sc22_df['subject'] == 'Biology I'), 'tested'].sum()
bio1_11_tested_sc22 = sc22_df.loc[(sc22_df['student_group'] == 'All Students') & (sc22_df['grade'] == '11') & (sc22_df['subject'] == 'Biology I'), 'tested'].sum()
bio1_12_tested_sc22 = sc22_df.loc[(sc22_df['student_group'] == 'All Students') & (sc22_df['grade'] == '12') & (sc22_df['subject'] == 'Biology I'), 'tested'].sum()

bio1_data = {
    'Biology I': ['9th Grade', '10th Grade', '11th Grade', '12th Grade'], '2021': [bio1_9_tested_sc21, bio1_10_tested_sc21, bio1_11_tested_sc21, bio1_12_tested_sc21], 'Biology I': ['9th Grade', '10th Grade', '11th Grade', '12th Grade'], '2022': [bio1_9_tested_sc22, bio1_10_tested_sc22, bio1_11_tested_sc22, bio1_12_tested_sc22]
}

bio1_testers_df = pd.DataFrame(bio1_data)

# Calculate the percentage change in test takers for Biology I
bio1_testers_df['Percent Change'] = (bio1_testers_df['2022'] - bio1_testers_df['2021']) / bio1_testers_df['2021'] * 100
bio1_testers_df['Percent Change'] = bio1_testers_df['Percent Change'].apply(lambda x: f'{x:.1f}%')

bio1_testers_df

,Biology I,2021,2022,Percent Change
0,9th Grade,24986,26785,7.2%
1,10th Grade,31001,36615,18.1%
2,11th Grade,6211,8762,41.1%
3,12th Grade,727,1013,39.3%


In [119]:
# The data has supressed rows where percentages for are below 1% or above 99% for the Below, Approaching, etc categories. These rows will be removed. 'pct_met_expected' is the aggregate percentage of valid tests that met expectations and exceeded expecatations 